<div style="display: flex; justify-content: center;">
    <img src="data/bane.png" style="border-radius: 20px; width: 60%;">
</div>

## Este Jupyter es una copia, editada, del Jupyter 'Limpieza_train'.

## LIBRERÍAS USADAS.


In [1]:
import pandas as pd 
from scipy.stats import f_oneway 
import seaborn as sns
import matplotlib.pyplot as plt 
from scipy.stats import pearsonr
import numpy as np 
import scipy.stats as stats 
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning) 
pd.set_option('display.max_rows', None) #Con esto quitamos el limite y vemos todas las filas.
pd.set_option('display.max_columns', None)# Ajustar las opciones de pandas para mostrar todas las columnas

from sklearn.preprocessing import OneHotEncoder , StandardScaler

from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer 
from sklearn.preprocessing import LabelEncoder

## CARGA DE DATOS.

In [2]:
df = pd.read_csv("./data/test.csv") 


## PEQUEÑA VISUALIZACIÓN DE LOS DATASETS.

In [3]:
df.head(5)

,id,laptop_ID,Company,Product,TypeName,Inches,ScreenResolution,Cpu,Ram,Memory,Gpu,OpSys,Weight
0,181,1098,HP,Spectre x360,Ultrabook,13.3,IPS Panel 4K Ultra HD 3840x2160,Intel Core i7 7500U 2.7GHz,16GB,512GB SSD,Intel HD Graphics 620,Windows 10,1.3kg
1,708,330,Acer,Aspire 5,Notebook,15.6,1366x768,AMD A12-Series 9720P 2.7GHz,8GB,256GB SSD,AMD Radeon RX 540,Windows 10,2.2kg
2,862,1260,Acer,Aspire ES1-572,Notebook,15.6,1366x768,Intel Core i3 6006U 2.0GHz,4GB,500GB HDD,Intel HD Graphics 520,Linux,2.4kg
3,1064,1137,HP,EliteBook 1040,Notebook,14.0,Full HD 1920x1080,Intel Core i5 6200U 2.3GHz,8GB,256GB SSD,Intel HD Graphics 520,Windows 7,1.43kg
4,702,1015,HP,ENVY -,Notebook,13.3,IPS Panel Full HD 1920x1080,Intel Core i5 7200U 2.5GHz,8GB,256GB SSD,Intel HD Graphics 620,Windows 10,1.34kg


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 391 entries, 0 to 390
Data columns (total 13 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   id                391 non-null    int64  
 1   laptop_ID         391 non-null    int64  
 2   Company           391 non-null    object 
 3   Product           391 non-null    object 
 4   TypeName          391 non-null    object 
 5   Inches            391 non-null    float64
 6   ScreenResolution  391 non-null    object 
 7   Cpu               391 non-null    object 
 8   Ram               391 non-null    object 
 9   Memory            391 non-null    object 
 10  Gpu               391 non-null    object 
 11  OpSys             391 non-null    object 
 12  Weight            391 non-null    object 
dtypes: float64(1), int64(2), object(10)
memory usage: 39.8+ KB


>Vemos que no hay nulos

## ESTUDIO/PREPARACION DE CADA COLUMNA.

## 1ª columna: "id".

In [5]:
#df['id'].value_counts()

>No hay ninguna entrada repetida. Esto es importante en una columna con identificadores, no es necesario hacer nada a esta columna.

## 2ª columna: "laptop_ID".

In [6]:
#df['laptop_ID'].value_counts()

En el jupyter 'train.csv' la eliminamos, aquí tambien.

In [7]:
df.drop(columns=['laptop_ID'], inplace=True) 


## 3ª columna: "Company".

In [8]:
#df['Company'].value_counts()


## 4ª columna: "Product".

In [9]:
df['Product'].value_counts()

Product
XPS 13                                        10
Legion Y520-15IKBN                             8
Inspiron 3567                                  8
Latitude 5580                                  7
ProBook 450                                    7
Inspiron 5570                                  7
Alienware 17                                   5
IdeaPad Y700-15ISK                             4
IdeaPad 320-17IKB                              4
Inspiron 5567                                  4
Vostro 3568                                    3
Latitude 5480                                  3
Satellite Pro                                  3
EliteBook 840                                  3
Inspiron 7570                                  3
ThinkPad Yoga                                  3
Spectre x360                                   3
Inspiron 5770                                  3
Chromebook 11                                  3
Precision 3520                                 3
ThinkPad X1 

> Vemos que hay entradas , veamos esto.

In [10]:
df_producto = df.loc[df['Product'] == 'Inspiron 3567'] 
df_producto.head(5)

,id,Company,Product,TypeName,Inches,ScreenResolution,Cpu,Ram,Memory,Gpu,OpSys,Weight
77,59,Dell,Inspiron 3567,Notebook,15.6,1366x768,Intel Core i3 6006U 2.0GHz,4GB,1TB HDD,AMD Radeon R5 M430,Linux,2.25kg
99,810,Dell,Inspiron 3567,Notebook,15.6,1366x768,Intel Core i5 7200U 2.5GHz,12GB,1TB HDD,Intel HD Graphics 620,Windows 10,2.25kg
135,1081,Dell,Inspiron 3567,Notebook,15.6,1366x768,Intel Core i5 7200U 2.5GHz,4GB,500GB HDD,AMD Radeon R5 M430,Windows 10,2.25kg
169,153,Dell,Inspiron 3567,Notebook,15.6,1366x768,Intel Core i3 6006U 2.0GHz,4GB,1TB HDD,Intel HD Graphics 520,Linux,2.25kg
180,514,Dell,Inspiron 3567,Notebook,15.6,Full HD 1920x1080,Intel Core i3 6006U 2GHz,4GB,1TB HDD,AMD Radeon R5 M430,Windows 10,2.2kg


>Se ha hecho lo mismo con otros modelos repetidos (Sin informacion entre parentesis en la columuna 'Product'). Para evitar que se alargue demasiado, se han eliminado al comprobar que se trata de ordenadores distintos.

In [11]:
#df[df['Product'].str.contains(r'\(')] #Lo comentamos, ya lo hemos visto.


>Vimos que algunas entradas de esta columna tienen información entre paréntesis y ademas estan repetidos, veamos si se trata de ordenadores repetidos y lo podemos eliminar. 


### Como comprobación, eliminaremos las columnas de ID y veremos si hay filas repetidas.

In [12]:
df_aux = df.drop(columns=['id']) 
columnas_duplicadas = df_aux.columns[df_aux.columns.duplicated()]
columnas_duplicadas


Index([], dtype='object')

### Ahora eliminemos los paréntesis.

In [13]:
df['Product'] = df['Product'].str.split('(').str.get(0)


### Veamos si podemos simplificar aun más.

In [14]:
#df['Product'].value_counts() #lo comento para no hacer demasido largo esto.

In [15]:
df['Modelo'] = df['Product'].apply(lambda x: x.split(' ')[0])
df['Submodelo'] = df['Product'].apply(lambda x: ' '.join(x.split(' ')[1:]) if len(x.split(' ')) > 1 else None)

In [16]:
for index, row in df.iterrows():
    # Si la entrada en la columna 'Company' no es 'HP', pasamos a la siguiente fila
    if row['Company'] != 'HP':
        continue
# Si la entrada en la columna 'Company' es 'HP' y la entrada en la columna 'Product' comienza con dos caracteres numéricos seguidos de '-', procesamos
    if row['Product'][:2].isdigit() and '-' in row['Product']:
        # Dividimos la entrada en la columna 'Product' en dos partes en el primer '-'
        parts = row['Product'].split('-', 1)
        # Asignamos los dos primeros caracteres numéricos a 'Modelo' y el resto a 'Submodelo'
        df.at[index, 'Modelo'] = parts[0][:2]
        df.at[index, 'Submodelo'] = parts[1]
    else:
        # Si no cumple las condiciones anteriores, no hacemos ningún cambio en esa fila
        continue


In [17]:
for index, row in df.iterrows():
    
    if row['Company'] != 'Lenovo':
        continue
# Si la entrada en la columna 'Company' es 'HP' y la entrada en la columna 'Product' comienza con dos caracteres numéricos seguidos de '-', procesamos
    if row['Product'][:2].isdigit() and '-' in row['Product']:
        # Dividimos la entrada en la columna 'Product' en dos partes en el primer '-'
        parts = row['Product'].split('-', 1)
        # Asignamos los dos primeros caracteres numéricos a 'Modelo' y el resto a 'Submodelo'
        df.at[index, 'Modelo'] = parts[0][:2]
        df.at[index, 'Submodelo'] = parts[1]
    else:
        # Si no cumple las condiciones anteriores, no hacemos ningún cambio en esa fila
        continue

In [18]:
asus_row = df[(df['Company'] == 'Asus') & (df['Product'] == 'A541NA-GO342 ')].index 

# Si encontramos la fila de Asus con ese producto, asignamos los valores correspondientes a Modelo y Submodelo
if not asus_row.empty:
    df.at[asus_row[0], 'Modelo'] = 'A541NA'
    df.at[asus_row[0], 'Submodelo'] = 'GO342'

In [19]:
for index, row in df.iterrows():
    # Si la entrada en la columna 'Company' no es 'Lenovo', pasamos a la siguiente fila
    if row['Company'] != 'Lenovo':
        continue
    
    # Si la entrada en la columna 'Product' comienza con 'B51' seguido de '-', procesamos
    if row['Product'].startswith('B51-'):
        # Dividimos la entrada en la columna 'Product' en dos partes en el primer '-'
        parts = row['Product'].split('-', 1)
        # Asignamos lo que está antes del '-' a 'Modelo' y lo que está después del '-' a 'Submodelo'
        df.at[index, 'Modelo'] = parts[0]
        df.at[index, 'Submodelo'] = parts[1]
    else:
        # Si no cumple las condiciones anteriores, no hacemos ningún cambio en esa fila
        continue

In [20]:
for index, row in df.iterrows():
    # Si la entrada en la columna 'Company' no es 'Asus', pasamos a la siguiente fila
    if row['Company'] != 'Asus':
        continue
    
    # Dividimos la entrada en la columna 'Product' en dos partes en el primer '-'
    parts = row['Product'].split('-', 1)
    
    # Verificamos si la lista parts tiene al menos dos elementos
    if len(parts) >= 2:
        # Asignamos la primera parte como 'Modelo' y la segunda parte como 'Submodelo'
        df.at[index, 'Modelo'] = parts[0]
        df.at[index, 'Submodelo'] = parts[1]
    else:
        # Si no hay suficientes partes, asignamos toda la entrada a 'Modelo' y dejamos 'Submodelo' como None
        df.at[index, 'Modelo'] = row['Product']
        df.at[index, 'Submodelo'] = None

In [21]:
for index, row in df.iterrows():
    # Si la entrada en la columna 'Company' no es 'Lenovo', pasamos a la siguiente fila
    if row['Company'] != 'Lenovo':
        continue
    
    # Dividimos la entrada en la columna 'Product' en dos partes en el primer '-'
    parts = row['Product'].split('-', 1)
    
    # Verificamos si la lista parts tiene al menos dos elementos
    if len(parts) >= 2:
        # Asignamos la primera parte como 'Modelo' y la segunda parte como 'Submodelo'
        df.at[index, 'Modelo'] = parts[0]
        df.at[index, 'Submodelo'] = parts[1]
    else:
        # Si no hay suficientes partes, asignamos toda la entrada a 'Modelo' y dejamos 'Submodelo' como None
        df.at[index, 'Modelo'] = row['Product']
        df.at[index, 'Submodelo'] = None

In [22]:
for index, row in df.iterrows():
    # Si la entrada en la columna 'Company' no es 'Lenovo', pasamos a la siguiente fila
    if row['Company'] != 'Lenovo':
        continue
    
    # Verificamos si la entrada en la columna 'Product' contiene 'ThinkPad'
    if 'ThinkPad' in row['Product']:
        # Dividimos la entrada en la columna 'Product' en dos partes en el primer 'ThinkPad'
        parts = row['Product'].split('ThinkPad', 1)
        
        # Asignamos 'ThinkPad' a 'Modelo' y lo que está después de 'ThinkPad' a 'Submodelo'
        df.at[index, 'Modelo'] = 'ThinkPad'
        df.at[index, 'Submodelo'] = parts[1].strip()  # Eliminamos los espacios en blanco alrededor de la cadena
    else:
        # Si 'ThinkPad' no está en la entrada, no hacemos ningún cambio en esa fila
        continue

In [23]:
for index, row in df.iterrows():
    # Si la entrada en la columna 'Company' no es 'Asus', pasamos a la siguiente fila
    if row['Company'] != 'Asus':
        continue
    
    # Verificamos si la entrada en la columna 'Product' contiene 'VivoBook'
    if 'VivoBook' in row['Product']:
        # Dividimos la entrada en la columna 'Product' en dos partes en el primer 'VivoBook'
        parts = row['Product'].split('VivoBook', 1)
        
        # Asignamos 'VivoBook' a 'Modelo' y lo que está después de 'VivoBook' a 'Submodelo'
        df.at[index, 'Modelo'] = 'VivoBook'
        df.at[index, 'Submodelo'] = parts[1].strip()  # Eliminamos los espacios en blanco alrededor de la cadena
    else:
        # Si 'VivoBook' no está en la entrada, no hacemos ningún cambio en esa fila
        continue

In [24]:
# Iteramos sobre cada fila del DataFrame
for index, row in df.iterrows():
    # Si la entrada en la columna 'Company' no es 'Asus', pasamos a la siguiente fila
    if row['Company'] != 'Asus':
        continue
    
    # Verificamos si la entrada en la columna 'Product' contiene 'Zenbook'
    if 'ZenBook' in row['Product']:
        # Asignamos 'ZenBook' a 'Modelo'
        df.at[index, 'Modelo'] = 'ZenBook'
        # Asignamos el resto de la entrada en 'Product' a 'Submodelo'
        df.at[index, 'Submodelo'] = row['Product'].split('ZenBook', 1)[1].strip()  # Eliminamos los espacios en blanco alrededor de la cadena
    elif 'Zenbook' in row['Product']:
        # Asignamos 'Zenbook' a 'Modelo'
        df.at[index, 'Modelo'] = 'Zenbook'
        # Asignamos el resto de la entrada en 'Product' a 'Submodelo'
        df.at[index, 'Submodelo'] = row['Product'].split('Zenbook', 1)[1].strip()  # Eliminamos los espacios en blanco alrededor de la cadena
    else:
        # Si 'Zenbook' no está en la entrada, no hacemos ningún cambio en esa fila
        continue


In [25]:
# Iteramos sobre cada fila del DataFrame
for index, row in df.iterrows():
    # Si la entrada en la columna 'Company' no es 'Lenovo', pasamos a la siguiente fila
    if row['Company'] != 'Lenovo':
        continue
    
    # Verificamos si la entrada en la columna 'Product' contiene 'Yoga' pero no 'ThinkPad'
    if 'Yoga' in row['Product'] and 'ThinkPad' not in row['Product']:
        # Asignamos 'Yoga' a 'Modelo'
        df.at[index, 'Modelo'] = 'Yoga'
        # Asignamos el resto de la entrada en 'Product' a 'Submodelo'
        df.at[index, 'Submodelo'] = row['Product'].split('Yoga', 1)[1].strip()  # Eliminamos los espacios en blanco alrededor de la cadena
    else:
        # Si 'Yoga' no está en la entrada o si 'ThinkPad' está presente, no hacemos ningún cambio en esa fila
        continue


In [26]:
# Iteramos sobre cada fila del DataFrame
for index, row in df.iterrows():
    # Si la entrada en la columna 'Company' no es 'Lenovo', pasamos a la siguiente fila
    if row['Company'] != 'Lenovo':
        continue
    
    # Verificamos si la entrada en la columna 'Product' contiene 'Legion'
    if 'Legion' in row['Product']:
        # Asignamos 'legion' a 'Modelo' (en minúsculas)
        df.at[index, 'Modelo'] = 'legion'
        # Asignamos el resto de la entrada en 'Product' a 'Submodelo'
        df.at[index, 'Submodelo'] = row['Product'].split('Legion', 1)[1].strip()  # Eliminamos los espacios en blanco alrededor de la cadena
    else:
        # Si 'Legion' no está en la entrada, no hacemos ningún cambio en esa fila
        continue


In [27]:
df.replace(to_replace="Portégé", value="Portege", inplace=True)

In [28]:
for index, row in df.iterrows():
    # Si la entrada en la columna 'Company' no es 'Asus', pasamos a la siguiente fila
    if row['Company'] != 'Asus':
        continue
    
    # Verificamos si la entrada en la columna 'Product' contiene 'Rog' o 'ROG'
    if 'Rog' in row['Product'] or 'ROG' in row['Product']:
        # Asignamos 'Rog' o 'ROG' a 'Modelo' según la entrada en 'Product'
        if 'Rog' in row['Product']:
            df.at[index, 'Modelo'] = 'Rog'
        else:
            df.at[index, 'Modelo'] = 'ROG'
        # Asignamos el resto de la entrada en 'Product' a 'Submodelo'
        submodelo_index = max(row['Product'].find('Rog'), row['Product'].find('ROG')) + len('Rog')  # Encuentra la última ocurrencia de 'Rog' o 'ROG'
        df.at[index, 'Submodelo'] = row['Product'][submodelo_index:].strip()  # Elimina los espacios en blanco alrededor de la cadena
    else:
        # Si 'Rog' o 'ROG' no están en la entrada, no hacemos ningún cambio en esa fila
        continue

In [29]:
for index, row in df.iterrows():
    # Si la entrada en la columna 'Company' no es 'Lenovo', pasamos a la siguiente fila
    if row['Company'] != 'Lenovo':
        continue
    
    # Verificamos si la entrada en la columna 'Product' contiene 'Thinkpad' y no contiene 'Yoga'
    if 'Thinkpad' in row['Product'] and 'Yoga' not in row['Product']:
        # Asignamos 'Thinkpad' a 'Modelo'
        df.at[index, 'Modelo'] = 'Thinkpad'
        # Asignamos el resto de la entrada en 'Product' a 'Submodelo'
        submodelo_index = row['Product'].find('Thinkpad') + len('Thinkpad')
        df.at[index, 'Submodelo'] = row['Product'][submodelo_index:].strip()  # Eliminamos los espacios en blanco alrededor de la cadena
    else:
        # Si 'Thinkpad' no está en la entrada o si 'Yoga' está presente, no hacemos ningún cambio en esa fila
        continue

In [30]:
# Iteramos sobre cada fila del DataFrame
for index, row in df.iterrows():
    # Si la entrada en la columna 'Company' no es 'Lenovo', pasamos a la siguiente fila
    if row['Company'] != 'Lenovo':
        continue
    
    # Verificamos si la entrada en la columna 'Product' contiene 'IdeaPad'
    if 'IdeaPad' in row['Product']:
        # Asignamos 'IdeaPad' a 'Modelo'
        df.at[index, 'Modelo'] = 'IdeaPad'
        # Asignamos el resto de la entrada en 'Product' a 'Submodelo'
        submodelo_index = row['Product'].find('IdeaPad') + len('IdeaPad')
        df.at[index, 'Submodelo'] = row['Product'][submodelo_index:].strip()  # Eliminamos los espacios en blanco alrededor de la cadena
    else:
        # Si 'IdeaPad' no está en la entrada, no hacemos ningún cambio en esa fila
        continue


In [31]:
df['Submodelo'].fillna('Sin_submodelo', inplace=True)
df['Submodelo'].replace('none', 'Sin_submodelo', inplace=True)
df['Submodelo'].replace(' ', 'Sin_submodelo', inplace=True) 
df['Submodelo'] = df['Submodelo'].apply(lambda x: 'Sin_submodelo' if pd.isna(x) or x.lower() == 'none' else x)

In [32]:
df.replace(to_replace="Rog", value="ROG", inplace=True)

In [33]:
#df.at[, 'Modelo'] = ''
#df.at[, 'Submodelo'] = ''

In [34]:
df.at[132, 'Modelo'] = 'Ideapad'
df.at[132, 'Submodelo'] = '110'  

df.at[213, 'Modelo'] = 'Vivobook'
df.at[213, 'Submodelo'] = 'X541UV' 

df.at[314, 'Modelo'] = 'Chromebook'
df.at[314, 'Submodelo'] = 'R11' 


df.at[375, 'Modelo'] = 'Leopard'
df.at[375, 'Submodelo'] = 'Pro' 


df.at[325, 'Submodelo'] = 'sin_submodelo ' 

df.at[183, 'Modelo'] = 'Vivobook'
df.at[183, 'Submodelo'] = 'X555' 

df.at[213, 'Modelo'] = 'Vivobook'
df.at[213, 'Submodelo'] = 'X541UV-DM1217T' 

df.at[372, 'Modelo'] = 'Aspire'
df.at[372, 'Submodelo'] = '7' 

df.at[370, 'Modelo'] = 'Vivobook '
df.at[370, 'Submodelo'] = 'X751NV-TY001T' 

df.at[335, 'Modelo'] = 'Vivobook'
df.at[335, 'Submodelo'] = ' Flip' 

df.at[313, 'Modelo'] = 'Chromebook'
df.at[313, 'Submodelo'] = 'C202SA' 

df.at[216, 'Modelo'] = 'IdeaPad'
df.at[216, 'Submodelo'] = '320-15ISK' 

df.at[366, 'Modelo'] = 'Vivobook'
df.at[366, 'Submodelo'] = 'X556UJ-XO044T' 

df.at[277, 'Modelo'] = 'Aspire'
df.at[277, 'Submodelo'] = 'E5-774G' 

df.at[337, 'Modelo'] = 'Leopard'
df.at[337, 'Submodelo'] = 'Pro' 

df.at[190, 'Modelo'] = 'Apache'
df.at[190, 'Submodelo'] = 'Pro' 

df.at[92, 'Modelo'] = 'Leopard'
df.at[92, 'Submodelo'] = 'Pro' 

df.at[178, 'Modelo'] = 'ROG'
df.at[178, 'Submodelo'] = 'G701VO' 

df.at[176, 'Modelo'] = 'Spin 3 SP315-51'
df.at[176, 'Submodelo'] = '3 SP315-51' 


df.at[4, 'Submodelo'] = 'sin_submodelo' 


df.at[16, 'Submodelo'] = '1 SP111-31' 

df.at[68, 'Modelo'] = 'Inspiron'
df.at[68, 'Submodelo'] = '5767' 

df.at[69, 'Modelo'] = 'Apache'
df.at[69, 'Submodelo'] = 'Pro' 

df.at[67, 'Modelo'] = 'Titan'
df.at[67, 'Submodelo'] = 'sli' 

df.at[45, 'Modelo'] = 'Matebook'


df.at[76, 'Modelo'] = 'Ideapad'
df.at[76, 'Submodelo'] = '520-15IKBR' 

df.at[86, 'Modelo'] = 'Raider'
df.at[86, 'Submodelo'] = 'sin_submodelo' 

df.at[113, 'Modelo'] = 'Aspire'
df.at[113, 'Submodelo'] = 'ES1-523-84K7' 

df.at[139, 'Modelo'] = 'Vivobook'
df.at[139, 'Submodelo'] = 'X542UQ-GO005' 

df.at[380, 'Modelo'] = 'Leopard'
df.at[380, 'Submodelo'] = 'Pro GP62M' 

df.at[132, 'Modelo'] = 'Ideapad'
df.at[132, 'Submodelo'] = '110-15ACL' 

df.at[43, 'Modelo'] = 'Stealth'
df.at[43, 'Submodelo'] = 'Pro' 

df.at[128, 'Modelo'] = 'Stealth'
df.at[128, 'Submodelo'] = 'Pro'

df.at[112, 'Modelo'] = 'Ideapad'
df.at[112, 'Submodelo'] = '320-15IAP' 

df.at[271, 'Modelo'] = 'Ideapad'
df.at[271, 'Submodelo'] = '320-15ISK' 



In [35]:
filtered_df = df[df['Modelo'] == 'Ideapad 320']

# Selecciona solo las columnas 'Company', 'Product', 'Modelo' y 'Submodelo'
selected_columns_df = filtered_df[['TypeName','Company', 'Product', 'Modelo', 'Submodelo']]

selected_columns_df

,TypeName,Company,Product,Modelo,Submodelo


In [36]:
df['Product'] = df['Product'].str.lower()
df['Modelo'] = df['Modelo'].str.lower()
df['Submodelo'] = df['Submodelo'].str.lower()

In [37]:
#df[['Company','Product', 'Modelo', 'Submodelo']].sort_values(by='Modelo') #Lo comentamos para que no se haga tan largo

In [38]:
df['Modelo'].value_counts()

Modelo
inspiron           43
ideapad            37
thinkpad           27
probook            21
aspire             20
latitude           17
elitebook          16
vivobook           14
rog                14
15                 14
xps                11
legion             10
zenbook             9
yoga                8
spectre             7
alienware           7
chromebook          6
vostro              6
zbook               5
precision           5
macbook             4
leopard             4
omen                4
tecra               4
portege             4
250                 4
envy                3
surface             3
gl72m               3
v310                3
satellite           3
pavilion            3
gs63vr              3
255                 3
stream              2
v330                2
gram                2
17                  2
fx753vd             2
notebook            2
apache              2
gt72s               2
stealth             2
fx502vm             2
travelmate          2
nit

## 5ª columna: "TypeName".

In [39]:
df['TypeName'].value_counts()

TypeName
Notebook              205
Gaming                 67
Ultrabook              61
2 in 1 Convertible     36
Workstation            11
Netbook                11
Name: count, dtype: int64

## 6ª columna: "'Inches'".

In [40]:
#df['Inches'].value_counts()

## 7ª columna: "ScreenResolution".

In [41]:
#df['ScreenResolution'].value_counts()

> La columna 'ScreenResolution' tiene mucha información, por lo que la iremos dividiendo en distintas columnas.

In [42]:
df['Resolucion'] = df['ScreenResolution'].str.extract(r'(\d{3,4}x\d{3,4})') #Con esto creamos una columna con la resolución.

In [43]:
#df['Resolucion'].value_counts()

Dividiremos en dos la columna "Resolución" y la haremos numérica.

In [44]:
df[['Resolucion_X', 'Resolucion_Y']] = df['Resolucion'].str.split('x', expand=True)

# Convertimos las nuevas columnas en valores numéricos
df['Resolucion_X'] = pd.to_numeric(df['Resolucion_X'])
df['Resolucion_Y'] = pd.to_numeric(df['Resolucion_Y'])


In [45]:
df['Tactil'] = False #Se crea una columna con tadas las entradas a 0, o sea... false.
df.loc[df['ScreenResolution'].str.contains('touchscreen', case=False), 'Tactil'] = True #Si en la columna ScreenResolution aparece 'touchscreen' se cambiara a True, o sea, verdadero

In [46]:
df['PANEL IPS'] = False #Se crea una columna con tadas las entradas a 0, o sea... false.
df.loc[df['ScreenResolution'].str.contains('IPS Panel', case=False), 'PANEL IPS'] = True #Si en la columna ScreenResolution aparece 'IPS Panel' se cambiara a 1, o sea, verdadero

In [47]:
df['Retina Display'] = False #Se crea una columna con tadas las entradas a 0, o sea... false.
df.loc[df['ScreenResolution'].str.contains('Retina Display', case=False), 'Retina Display'] = True #Si en la columna ScreenResolution aparece 'Quad HD+' se cambiara a 1, o sea, verdadero

> Sabemos que hay 17 entradas de la marca Apple. La pantalla 'Retina' es de Apple.  
Si hay 17, podemos eliminarla, puesto que esta información ya aparece en la columna marca.

> Hay 3 ordenadores Apple sin pantalla Retina, por lo que no podemos eliminarla.

In [48]:
df['Full HD'] = 0 #Se crea una columna con tadas las entradas a 0, o sea... false.
df.loc[df['ScreenResolution'].str.contains('Full HD', case=False), 'Full HD'] = 1 #Si en la columna ScreenResolution aparece 'Full HD' se cambiara a 1, o sea, verdadero

In [49]:
df['4K Ultra HD'] = 0 #Se crea una columna con tadas las entradas a 0, o sea... false.
df.loc[df['ScreenResolution'].str.contains('4K Ultra HD', case=False), '4K Ultra HD'] = 1 #Si en la columna ScreenResolution aparece '4K Ultra HD' se cambiara a 1, o sea, verdadero

In [50]:
df['Quad HD+'] = 0 #Se crea una columna con tadas las entradas a 0, o sea... false.
df.loc[df['ScreenResolution'].str.contains('Quad HD+', case=False), 'Quad HD+'] = 1 #Si en la columna ScreenResolution aparece 'Quad HD+' se cambiara a 1, o sea, verdadero

Ahora crearemos una columna "Definicion" que diga "Full HD" , "4K Ultra HD" o "Quad HD+" y despues eliminaremos las 3 anteriores.

In [51]:
df['Definicion'] = ''
df.loc[df['Full HD'] == 1, 'Definicion'] = 'Full HD'
df.loc[df['4K Ultra HD'] == 1, 'Definicion'] = '4K Ultra HD'
df.loc[df['Quad HD+'] == 1, 'Definicion'] = 'Quad HD+'
df['Definicion'] = df['Definicion'].replace('', 'No especificado')


In [52]:
#df['Definicion'].value_counts()

La definicion (Full HD, 4k...) depende de la resolucion, vemaos si podemos eliminar la columna definicion.

In [53]:
df_full_hd = df[df['Definicion'] == 'Full HD'] 
df_full_hd['Resolucion'].value_counts()

Resolucion
1920x1080    259
1366x768       1
2160x1440      1
2560x1440      1
Name: count, dtype: int64

Esto puede ser un poco problematico, segun chatgtp full hd es solo 1920x1080

In [54]:
df_full_hd.loc[df_full_hd['Resolucion'].isin(['1920x1200', '2160x1440'])]

,id,Company,Product,TypeName,Inches,ScreenResolution,Cpu,Ram,Memory,Gpu,OpSys,Weight,Modelo,Submodelo,Resolucion,Resolucion_X,Resolucion_Y,Tactil,PANEL IPS,Retina Display,Full HD,4K Ultra HD,Quad HD+,Definicion
45,958,Huawei,matebook x,Ultrabook,13.0,IPS Panel Full HD 2160x1440,Intel Core i7 7500U 2.7GHz,8GB,512GB SSD,Intel HD Graphics 620,Windows 10,1.05kg,matebook,x,2160x1440,2160,1440,False,True,False,1,0,0,Full HD


In [55]:
kkkk= df[df['Definicion'] == '4K Ultra HD'] 
kkkk['Resolucion'].value_counts()

Resolucion
3840x2160    9
Name: count, dtype: int64

> Esto es bueno.

In [56]:
quad= df[df['Definicion'] == 'Quad HD+'] 
quad['Resolucion'].value_counts()

Resolucion
3200x1800    12
2560x1440     2
Name: count, dtype: int64

>Segun chaty ambos pueden ser Quad HD+

Ya podemos eliminar las columnas  las columnas temporales que creamos.

In [57]:
df.drop(columns=['Resolucion', 'Full HD', '4K Ultra HD', 'Quad HD+'], inplace=True) 



## 8ª columna: "Cpu".

In [58]:
#df['Cpu'].value_counts()

Al igual que hicimos con la columna 'ScreenResolution', dividiremos la información de la columna *'Cpu'* en distintas columnas.

In [59]:
#Primero la marca
df['Marca_Cpu'] = 'Samsung'  # Por defecto la marca será 'nisu'

df.loc[df['Cpu'].str.contains('Intel'), 'Marca_Cpu'] = 'Intel'
df.loc[df['Cpu'].str.contains('AMD'), 'Marca_Cpu'] = 'AMD'



In [60]:
df['Cpu'] = df['Cpu'].str.replace('.0GHz', 'GHz') 
 
df['Cpu'] = df['Cpu'].str.replace('Intel Core i5 7200U 2.70GHz', 'Intel Core i5 7200U 2.7GHz') 

df['Cpu'] = df['Cpu'].str.replace('Intel Celeron Dual Core N3060 1.60GHz', 'Intel Celeron Dual Core N3060 1.6GHz') 

df['Cpu'] = df['Cpu'].str.replace('Intel Core i7 6500U 2.50GHz', 'Intel Core i7 6500U 2.5GHz') 

df['Cpu'] = df['Cpu'].str.replace('Intel Core i5 7200U 2.50GHz', 'Intel Core i5 7200U 2.5GHz') 


df['Cpu'] = df['Cpu'].str.replace('Atom X5-Z8350', 'Atom x5-Z8350') 
df['Cpu'] = df['Cpu'].str.replace('Atom Z8350', 'Atom x5-Z8350') 
df['Cpu'] = df['Cpu'].str.replace('A6-Series A6-9220', 'A6-Series 9220') 
df['Cpu'] = df['Cpu'].str.replace('A9-Series A9-9420', 'A9-Series 9420') 
df['Cpu'] = df['Cpu'].str.replace('E-Series 9000', 'E-Series E2-9000') 

df['Cpu'] = df['Cpu'].str.replace('AMD A10-Series A10-9620P 2.5GHz', 'AMD A10-Series 9620P 2.5GHz')

In [61]:
#Ahora La velocidad.
df['Potencia_Cpu(GHz)'] = 'nisu' 
df['Potencia_Cpu(GHz)'] = df['Cpu'].str.extract(r'(\d+\.*\d*)\s*GHz')
df['Potencia_Cpu(GHz)'] = df['Potencia_Cpu(GHz)'].astype(float)
#df['Potencia_Cpu(GHz)'].value_counts() 

In [62]:
df['modelocpu_temporal'] = df['Cpu'].apply(lambda x: ' '.join(x.replace('AMD', '').replace('Intel', '').split()[:-1]) if 'GHz' in x else x) #esto es temporal, para facilitar las cosas
df[['Modelo_Cpu', 'Submodelo_Cpu']] = df['modelocpu_temporal'].str.split(n=1, expand=True)

In [63]:
df[['Modelo_Cpu', 'Submodelo_Cpu']] = df['modelocpu_temporal'].str.split(n=1, expand=True)

In [64]:
df[['Cpu','modelocpu_temporal','Marca_Cpu','Modelo_Cpu', 'Submodelo_Cpu','Potencia_Cpu(GHz)']].sort_values(by='modelocpu_temporal')

,Cpu,modelocpu_temporal,Marca_Cpu,Modelo_Cpu,Submodelo_Cpu,Potencia_Cpu(GHz)
206,AMD A10-Series 9600P 2.4GHz,A10-Series 9600P,AMD,A10-Series,9600P,2.40
182,AMD A10-Series 9600P 2.4GHz,A10-Series 9600P,AMD,A10-Series,9600P,2.40
39,AMD A10-Series 9620P 2.5GHz,A10-Series 9620P,AMD,A10-Series,9620P,2.50
1,AMD A12-Series 9720P 2.7GHz,A12-Series 9720P,AMD,A12-Series,9720P,2.70
274,AMD A12-Series 9720P 3.6GHz,A12-Series 9720P,AMD,A12-Series,9720P,3.60
256,AMD A4-Series 7210 2.2GHz,A4-Series 7210,AMD,A4-Series,7210,2.20
132,AMD A6-Series 7310 2GHz,A6-Series 7310,AMD,A6-Series,7310,2.00
158,AMD A6-Series 9220 2.5GHz,A6-Series 9220,AMD,A6-Series,9220,2.50
89,AMD A6-Series 9220 2.5GHz,A6-Series 9220,AMD,A6-Series,9220,2.50
188,AMD A6-Series 9220 2.5GHz,A6-Series 9220,AMD,A6-Series,9220,2.50


In [65]:
df['Submodelo_Cpu'].value_counts()

Submodelo_Cpu
i5 7200U           48
i7 7500U           46
i7 7700HQ          45
i7 8550U           29
i3 6006U           23
i5 8250U           18
i5 6200U           17
i7 6500U           16
i7 6700HQ          15
i5 7300HQ          14
Dual Core N3350    10
i3 7100U            8
Dual Core N3060     8
i5 7300U            7
i7 6600U            6
i7 6820HK           5
9420                5
i3 7130U            4
9220                4
i7 7600U            4
i5                  3
Dual Core N3050     3
i7 7820HQ           3
i5 7440HQ           3
9600P               2
i3 6100U            2
i5 6300U            2
M 6Y75              2
i5 6300HQ           2
i7 6820HQ           2
M 6Y30              2
7410                2
i7 7660U            2
Quad Core N3450     2
9720P               2
Quad Core N3710     2
Quad Core N4200     2
E2-9000e            2
i5 7Y57             1
i5 6260U            1
i7 8650U            1
Quad Core N3160     1
i5 6440HQ           1
7210                1
9410              

La columna 'modelocpu_temporal' ya ha cumplido su función, nos sirvió de gran ayuda... pero ya no la necesitamos, de hecho nos estorba, y como no tenemos sentimiento de apego con ella, pues al cielo con ella.

In [66]:
df.drop(columns=['modelocpu_temporal'], inplace=True) 



## 9ª columna: "Ram".

In [67]:
df['Ram'].value_counts()

Ram
8GB     185
4GB     101
16GB     75
12GB     10
6GB       9
2GB       6
32GB      3
64GB      1
24GB      1
Name: count, dtype: int64

> Para pasarlo a numérico necesitamos eliminar 'GB'.

In [68]:
df['Ram'] = df['Ram'].str.replace('GB', '')
df['Ram'] = pd.to_numeric(df['Ram'])
df = df.rename(columns={'Ram': 'Ram (GB)'})

In [69]:
df['Ram (GB)'].value_counts()

Ram (GB)
8     185
4     101
16     75
12     10
6       9
2       6
32      3
64      1
24      1
Name: count, dtype: int64

> Ya está en numérica 

## 10ª columna: "Memory".

In [70]:
#df['Memory'].value_counts()

Creamos una columna con los gigas del disco duro principal.

In [71]:
df['Memoria principal'] = df['Memory'].str.split().str[0] #Con esto pongo cuantos gb o tb tiene el disco duro princpial. 
df['Memoria principal'] = df['Memoria principal'].str.replace('GB', '') #Eliminamos 'GB' de las entradas donde aparece. 
df['Memoria_Principal'] = df['Memoria principal'] #Creamos una nueva columna para poder multiplicar por mil los teras.
df['Memoria_Principal'] = df['Memoria_Principal'].str.replace('TB', '') #Eliminamos 'TB' de las entradas donde aparece. 
df['Memoria_Principal'] = pd.to_numeric(df['Memoria_Principal'], errors='coerce') #Convertimos a numerico. 
df['Memoria_Principal'] = df['Memoria principal'].apply(lambda x: float(x.replace('TB', '').strip()) * 1000 if 'TB' in x else x) #Si son teras multiplicamos por mil 
df.drop(columns=['Memoria principal'], inplace=True) #Eliminamos la columna que ya no nos sirve.
df['Memoria_Principal'] = df['Memoria_Principal'].astype(float) #Convertimos a numerico.
df = df.rename(columns={'Memoria_Principal': 'Memoria_Principal(GB)'}) #cambiamos el nombre




Creamos una columna con el tipo del disco duro principal

In [72]:
df['Tipo_mem_principal'] = df['Memory'].apply(lambda x: x.split(" ")[1].strip() if "+" in x else x.split(" ", 1)[1].strip()) #Creamos una columna cone l primer tipo de memoria que sale.
df['Tipo_mem_principal'] = df['Tipo_mem_principal'].replace("Flash", "Flash Storage") #Arreglamos un pequeño fallito


In [73]:
df['Memoria_Principal(GB)'].value_counts()

Memoria_Principal(GB)
256.0     148
1000.0     80
128.0      57
512.0      49
500.0      32
32.0       12
2000.0      5
64.0        4
16.0        3
508.0       1
Name: count, dtype: int64

In [74]:
df['Tipo_mem_principal'].value_counts()

Tipo_mem_principal
SSD              256
HDD              111
Flash Storage     21
Hybrid             3
Name: count, dtype: int64

Creamos una columna con los gigas que tiene el segundo disco, 0 si no tiene

In [75]:
df['Segundo disco'] = df['Memory'].apply(lambda x: x[x.find('+')+1:].split()[0] if '+' in x else 0) #Si no aparece el + se pone "0" en la columna nueva, si aparece se pone su memoria
mask = df['Segundo disco'].str.contains('GB', na=False)#Eliminamos los caracteres "GB"
df.loc[mask, 'Segundo disco'] = df.loc[mask, 'Segundo disco'].str.replace(r'(\d+)GB', r'\1', regex=True) #Eliminamos los caracteres "GB" 
df['Segundo_Disco'] = df['Segundo disco'] #CReamos una columna igual. 
mask = df['Segundo_Disco'].str.contains('TB', na=False)#Eliminamos los caracteres "TB" 
df.loc[mask, 'Segundo_Disco'] = df.loc[mask, 'Segundo_Disco'].str.replace(r'(\d+)TB', r'\1', regex=True) #Eliminamos los caracteres "TB" 
df['Segundo_Disco'] = pd.to_numeric(df['Segundo_Disco'], errors='coerce') #Convertimos a numerico. 
mask = df['Segundo disco'].str.contains('TB', na=False) #Buscamos aquellos que vienen en teras
df.loc[mask, 'Segundo_Disco'] *= 1000 #Multipicamos por 1000 
df = df.rename(columns={'Segundo_Disco': 'Memoria_Secundario(GB)'}) 
df.drop(columns=['Segundo disco'], inplace=True) 



In [76]:
df['Memoria_Secundario(GB)'].value_counts()

Memoria_Secundario(GB)
0       323
1000     58
2000      8
256       2
Name: count, dtype: int64

Creamos una columna con el tipo del disco duro secundario.

In [77]:
df['Tipo_mem_secundario'] = 'Sin disco secundario'

mask = df['Memory'].str.contains(r'\+', na=False)

df.loc[mask, 'Tipo_mem_secundario'] = df.loc[mask, 'Memory'].str.split().str[-1]




In [78]:
df['Tipo_mem_secundario'].value_counts()

Tipo_mem_secundario
Sin disco secundario    323
HDD                      66
SSD                       2
Name: count, dtype: int64

Dejamos el codigo para liminar la columna "Memory" preparada

In [79]:
#df.drop(columns=['Memory'], inplace=True) 

## 11ª columna: "Gpu". 
>Al igual que hicimos con la columna 'Cpu' aquí también la dividiremos en otras columnas.

Primero la marca (es lo más fácil).

In [80]:
df['Marca_Gpu'] = df['Gpu'].apply(lambda x: 'Nvidia' if 'nvidia' in x.lower() else ('Intel' if 'intel' in x.lower() else ('AMD' if 'amd' in x.lower() else 'Otro')))

In [81]:
#df[['Gpu','Marca_Gpu']].sort_values(by='Gpu')

In [82]:
#Vemos que aparece 'Nvidia GeForce GTX 960<U+039C>' eso de '<U+039C>' no significa nada, asi que lo quitamos. 
df['Gpu'] = df['Gpu'].str.replace('Nvidia GeForce GTX 960<U+039C>', 'Nvidia GeForce GTX 960') 

In [83]:
#df['Marca_Gpu'].value_counts()

Ahora hagamos una columna temporal, pa' que sea más fácil sacar información... Sin la marca.

In [84]:
df['Temporal_Gpu'] = df['Gpu'].apply(lambda x: x.lower().replace('nvidia', '').replace('intel', '').replace('amd', ''))


In [85]:
#df[['Gpu','Marca_Gpu', 'Temporal_Gpu']].sort_values(by='Temporal_Gpu')

Pasándole la lista temporal a nuestro amigo Chaty, veremos si hay algunas que son la misma pero escritas de forma distinta.







In [86]:
df['Gpu'] = df['Gpu'].str.replace('hd graphics 620', 'Hd graphics 620') 
df['Gpu'] = df['Gpu'].str.replace('geforce gtx 940mx', 'geforce gtx 940MX') 

In [87]:
#df['Temporal_Gpu'].value_counts() 
#Según chatgtp parece estar bien.

Ahora si, Modelo y Submodelo.

In [88]:
# Crear las columnas Modelo_Gpu y Submodelo_Gpu
df['Modelo_Gpu'] = ''
df['Submodelo_Gpu'] = ''

# Iterar sobre las filas del DataFrame
for index, row in df.iterrows():
    # Dividir la cadena en palabras
    palabras = row['Temporal_Gpu'].split()
    
    # Si hay solo una palabra
    if len(palabras) == 1:
        df.at[index, 'Modelo_Gpu'] = palabras[0]
        df.at[index, 'Submodelo_Gpu'] = 'no_especificado'
    else:
        # Tomar la primera palabra como el modelo
        df.at[index, 'Modelo_Gpu'] = palabras[0]
        
        # Si la primera palabra es "hd graphics" y hay más palabras después de "hd graphics"
        if palabras[0] == 'hd' and palabras[1] == 'graphics':
            df.at[index, 'Modelo_Gpu'] = 'hd graphics'
            if len(palabras) > 2:
                df.at[index, 'Submodelo_Gpu'] = palabras[2]
            else:
                df.at[index, 'Submodelo_Gpu'] = 'no_especificado'
        elif palabras[0] == 'uhd' and palabras[1] == 'graphics':
            df.at[index, 'Modelo_Gpu'] = 'uhd graphics'
            if len(palabras) > 2:
                df.at[index, 'Submodelo_Gpu'] = palabras[2]
            else:
                df.at[index, 'Submodelo_Gpu'] = 'no_especificado'
        # Si la segunda palabra es un número, tomarla como el submodelo
        elif palabras[1].isdigit():
            df.at[index, 'Submodelo_Gpu'] = palabras[1]
        else:
            # Tomar todas las palabras restantes como el submodelo
            df.at[index, 'Submodelo_Gpu'] = ' '.join(palabras[1:])

    # Manejar casos especiales
    if row['Temporal_Gpu'] == 'r17m-m1-70':
        df.at[index, 'Modelo_Gpu'] = 'radeon'
        df.at[index, 'Submodelo_Gpu'] = 'r7 M530'
    elif 'sli' in row['Temporal_Gpu']:
        df.at[index, 'Submodelo_Gpu'] = row['Temporal_Gpu']







In [89]:
#gtx 1050ti 
df['Submodelo_Gpu'] = df['Submodelo_Gpu'].str.replace('gtx 1050ti', 'gtx 1050 ti') 

In [90]:
#df[['Gpu','Marca_Gpu', 'Modelo_Gpu','Submodelo_Gpu']].sort_values(by='Modelo_Gpu')

In [91]:
#df['Submodelo_Gpu'].value_counts()

Ahora eliminamos la temporal.

In [92]:
df.drop(columns=['Temporal_Gpu'], inplace=True)

In [93]:
#df[['Gpu','Marca_Gpu','Modelo_Gpu', 'Submodelo']].sort_values(by='Modelo_Gpu')

## 12ª columna: "'OpSys'".

In [94]:
#df['OpSys'].value_counts()

## 13ª columna: "Weight".

In [95]:
#df['Weight'].value_counts()

> Para pasarlo a numérico necesitamos eliminar 'Kg'.

In [96]:
df['Weight'] = df['Weight'].str.replace('kg', '')
df['Weight'] = pd.to_numeric(df['Weight'])

> Ya está en numérica 

In [97]:
df = df.rename(columns={'Weight': 'Weight (Kg)'})

Nulos?

In [98]:
df.dropna(inplace=True)

### Codificacion:

columns_to_encode = ['Company', 'Product', 'TypeName', 'ScreenResolution', 'Cpu', 'Memory', 'Gpu', 'OpSys', 'Modelo', 'Submodelo', 'Tactil', 'PANEL IPS', 'Retina Display', 'Definicion', 'Marca_Cpu', 'Modelo_Cpu', 'Submodelo_Cpu', 'Tipo_mem_principal', 'Tipo_mem_secundario', 'Marca_Gpu', 'Modelo_Gpu', 'Submodelo_Gpu']


TIPO ORDENADOR

le = LabelEncoder()
le.fit(df['TypeName'])
df["TypeName_encode"] = le.transform(df['TypeName'])
df.drop(columns=['TypeName'], inplace=True)

MARCA

le.fit(df['Company'])
df["Company_encode"] = le.transform(df['Company'])
#df.drop(columns=['Company'], inplace=True)

mapping = {5:6,6:7,7:8,8:9,9:10,10:11,11:13,12:14,13:15,14:16,15:17,16:18 }

# Reemplazar valores en la columna "Company_encode" usando el diccionario de mapeo
df["Company_encode"] = df["Company_encode"].replace(mapping)


Sistema operativo

le.fit(df['OpSys'])
df["OpSys_encode"] = le.transform(df['OpSys'])
df.drop(columns=['OpSys'], inplace=True)

mapping = {0:1,1:2,2:3,3:4,4:5,5:6,6:7,7:8}

# Reemplazar valores en la columna "Company_encode" usando el diccionario de mapeo
df["OpSys_encode"] = df["OpSys_encode"].replace(mapping)

Tactil

le.fit(df['Tactil'])
df["Tactil_encode"] = le.transform(df['Tactil'])
df.drop(columns=['Tactil'], inplace=True)

PANEL IPS

le.fit(df['PANEL IPS'])
df["PANEL IPS_encode"] = le.transform(df['PANEL IPS'])
df.drop(columns=['PANEL IPS'], inplace=True)

Retina Display 

le.fit(df['Retina Display'])
df["Retina Display_encode"] = le.transform(df['Retina Display'])
#df.drop(columns=['Retina Display'], inplace=True)

Definicion  

le.fit(df['Definicion'])
df["Definicion_encode"] = le.transform(df['Definicion'])
df.drop(columns=['Definicion'], inplace=True)

Marca_Cpu

le.fit(df['Marca_Cpu'])
df["Marca_Cpu_encode"] = le.transform(df['Marca_Cpu'])
#df.drop(columns=['Marca_Cpu'], inplace=True)


Tipo_mem_principal

le.fit(df['Tipo_mem_principal'])
df["Tipo_mem_principal_encode"] = le.transform(df['Tipo_mem_principal'])
df.drop(columns=['Tipo_mem_principal'], inplace=True)

Tipo_mem_secundario

le.fit(df['Tipo_mem_secundario'])
df["Tipo_mem_secundario_encode"] = le.transform(df['Tipo_mem_secundario'])
df.drop(columns=['Tipo_mem_secundario'], inplace=True)

mapping = {2:3,1:2 }


df["Tipo_mem_secundario_encode"] = df["Tipo_mem_secundario_encode"].replace(mapping)

Marca_Gpu

unique_entries = df[["Tipo_mem_secundario_encode", 'Tipo_mem_secundario']].drop_duplicates(subset=['Tipo_mem_secundario']).sort_values(by='Tipo_mem_secundario')

# Muestra el resultado
print(unique_entries)

In [99]:
"""
encoder = OneHotEncoder()

# Codificar las columnas seleccionadas
encoded_columns = encoder.fit_transform(df[columns_to_encode])

# Obtener los nombres de las nuevas columnas
encoded_column_names = encoder.get_feature_names_out(columns_to_encode)

# Crear un nuevo dataframe con las columnas originales y las codificadas
df = pd.concat([df.drop(columns_to_encode, axis=1), pd.DataFrame(encoded_columns.toarray(), columns=encoded_column_names)], axis=1)
"""




'\nencoder = OneHotEncoder()\n\n# Codificar las columnas seleccionadas\nencoded_columns = encoder.fit_transform(df[columns_to_encode])\n\n# Obtener los nombres de las nuevas columnas\nencoded_column_names = encoder.get_feature_names_out(columns_to_encode)\n\n# Crear un nuevo dataframe con las columnas originales y las codificadas\ndf = pd.concat([df.drop(columns_to_encode, axis=1), pd.DataFrame(encoded_columns.toarray(), columns=encoded_column_names)], axis=1)\n'


columnas_numericas = ['Inches','Ram (GB)','Weight (Kg)','Resolucion_X','Resolucion_Y','Potencia_Cpu(GHz)','Memoria_Principal(GB)','Memoria_Secundario(GB)']

scaler = StandardScaler()
df[columnas_numericas] = scaler.fit_transform(df[columnas_numericas])

In [100]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 391 entries, 0 to 390
Data columns (total 31 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   id                      391 non-null    int64  
 1   Company                 391 non-null    object 
 2   Product                 391 non-null    object 
 3   TypeName                391 non-null    object 
 4   Inches                  391 non-null    float64
 5   ScreenResolution        391 non-null    object 
 6   Cpu                     391 non-null    object 
 7   Ram (GB)                391 non-null    int64  
 8   Memory                  391 non-null    object 
 9   Gpu                     391 non-null    object 
 10  OpSys                   391 non-null    object 
 11  Weight (Kg)             391 non-null    float64
 12  Modelo                  391 non-null    object 
 13  Submodelo               391 non-null    object 
 14  Resolucion_X            391 non-null    in

### Ahora guardaremos el dataframe ya limpio y preparado (aunque no del todo) en un archivo CSV, el cual usaremos en otro Jupyter para el modelado.







In [101]:
df.to_csv('data/limpios/df_test_prueba_larga.csv', index=False)